# Generating team and salaries data

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.ticker as ticker
from scipy.stats import linregress
import seaborn as sns
import numpy as np
import sklearn
import scipy.stats as stats


In [ ]:
#Teams data import

teams_df = pd.read_csv("Baseball Dataset\Teams.csv")
                          
teams_df.head()

#Limiting data to years 1985 - 2015
teams_df = teams_df[teams_df['yearID']>=1985].reset_index(drop=True)

teams_df.head()

teams_df.groupby('franchID').count()

#Limiting to franchise in full sample
    #Note: The Arizona Diamondbacks, Colorado Rockies, Flordia Marlins, and Tampa Bay Devils franchises 
    #did not exist during the full study time perion (1985-2015) and are dropped from the dataset

excluded_franchID = ['ARI','COL','FLA','TBD']

teams_df = teams_df.loc[~teams_df['franchID'].isin(excluded_franchID)]

teams_df.groupby('franchID').count()


In [ ]:
#Salaries data import

salaries_df = pd.read_csv("Baseball Dataset\Salaries.csv")
                          
salaries_df.head()



In [ ]:
#Team player salaries summary statistics
team_salary_stats = salaries_df.groupby(['yearID','teamID']).agg(total_player_salaries = ('salary', 'sum'),
                                                                  med_player_salary = ('salary','median'),
                                                                  min_player_salary = ('salary','min'),
                                                                  max_player_salary = ('salary','max'),
                                                                  salary_var = ('salary','var'))

team_salary_stats.head()

In [ ]:
#Checking # of teams per year
team_salary_stats.groupby('yearID').nunique()

#Consider restricting our sample to original 26 teams


In [ ]:
#Merging Team and Salary Data
team_salary_df = pd.merge(teams_df, team_salary_stats, on = ('yearID','teamID'))

team_salary_df.head() 

# Export the merged data into a csv
team_salary_df.to_csv("Output/team_salary_merged.csv", index=False)
    

In [ ]:
#Adding CBT thresholds and flags for 2003-2015

thresholds = pd.read_csv("Salary Thresholds/2003-2015 Salary Thresholds.csv", thousands=',')

team_salary_2003_2015 = pd.merge(team_salary_df,thresholds, left_on='yearID', right_on='Year')

team_salary_2003_2015['over_threshold'] =np.where(team_salary_2003_2015['total_player_salaries']>team_salary_2003_2015['Threshold'], 1,0)

team_salary_2003_2015['times_over_threshold'] = team_salary_2003_2015.groupby('franchID')['over_threshold'].cumsum()

team_salary_2003_2015.head()

In [ ]:
#Dataset info
team_salary_2003_2015.info() 

team_salary_2003_2015.describe()

In [ ]:
rank_by_salary_df = teams_df.copy(deep=True)
rank_by_salary_df["Salary Rank"] = ""
rank_by_salary_df.head()

In [ ]:
# adding a salary difference column
diff_by_salary_df = team_salary_df.copy(deep=True)
diff_by_salary_df["Salary Rank"] = ""
diff_by_salary_df.head()

In [ ]:
team_salary_df['Salary Rank'] = team_salary_df.groupby("yearID")["total_player_salaries"].rank(ascending=False).astype(int)

team_salary_df.head()

In [ ]:
#Merging team, salary, and salary rank data

#yearly_team_salaries = yearly_team_salaries.drop(columns='total_player_salaries')

team_salary_df['Salary Rank'] = team_salary_df.groupby("yearID")["total_player_salaries"].rank(ascending=False).astype(int)

#Merging salary thresholds for 2003-2015

thresholds = pd.read_csv("Salary Thresholds/2003-2015 Salary Thresholds.csv", thousands=',')

team_salary_tax_df = pd.merge(team_salary_df,thresholds, left_on='yearID', right_on='Year', how='left').drop(columns=["Year"])

team_salary_tax_df['Threshold'].fillna(0,inplace=True)

#Creating Taxed (over_threshold) Flag for all years
top_5_years = [1997,1998,1999,2002]

team_salary_tax_df['over_threshold'] = np.where(np.logical_xor(np.logical_and(team_salary_tax_df['yearID'].isin(top_5_years), team_salary_tax_df['Salary Rank'] <=5),
                                                               np.logical_and(team_salary_tax_df['yearID'] >=2003, team_salary_tax_df['total_player_salaries']>team_salary_tax_df['Threshold'])),
                                                 1,0)
     #for 1997-2002 top 5 spending teams were taxed
     #for 2003-2015 teams were taxed if spent over threshold


team_salary_tax_df.head()



In [ ]:
#Counting # of times taxed since 1997
team_salary_tax_df['times_over_threshold'] = team_salary_tax_df.groupby('franchID')['over_threshold'].cumsum()

# Export the merged data into a csv
team_salary_tax_df.to_csv("Output/team_salary_tax.csv", index=False)

#Sorting by most to less taxed teams

df_2015 = team_salary_tax_df[team_salary_tax_df['yearID']==2015]
times_taxed = df_2015[['franchID', 'name', 'times_over_threshold']].sort_values('times_over_threshold', ascending=False)

times_taxed



# Question 1 Analysis, salary impacts (Clayton Knight )

In [ ]:
# group by year and franchID, and calculate the total player salaries for each group
yearly_team_salaries = team_salary_df.groupby(["yearID", "franchID"])["total_player_salaries"].sum().reset_index()

# Rank the total player salaries within each year
yearly_team_salaries['Salary Rank'] = yearly_team_salaries.groupby("yearID")["total_player_salaries"].rank(ascending=False).astype(int)

yearly_team_salaries.head(-30)

yearly_team_salaries[yearly_team_salaries["Salary Rank"]==1]

In [ ]:
# Rename the columns
yearly_team_salaries_rename = yearly_team_salaries.rename(columns={'yearID': 'Year', 'franchID': 'Franchise ID', 'total_player_salaries': 'Total Team Salary'})

yearly_team_salaries_rename

In [ ]:
# getting the difference between top and bottom each year
yearly_salary_difference = yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].max() - yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].min()

# getting the average salary per year

avg_salary_per_year = yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].mean()

# bar chart
plt.figure(figsize=(10,6))
bar_colors=["blue" if year != 1997 else "red" for year in yearly_salary_difference.index]
yearly_salary_difference.plot(kind="bar", color=bar_colors)
plt.title("Disparity Between Teams with Highest and Lowest Salaries per Year")
plt.xlabel("Year")
plt.ylabel("Salary Difference, USD")
plt.xticks(rotation=45)
plt.tight_layout
formatter = ticker.StrMethodFormatter('{x:,.0f}')
plt.gca().yaxis.set_major_formatter(formatter)
plt.savefig("Output/Disparity Fig.png")
plt.show()

In [ ]:
avg_salary_per_year = yearly_team_salaries_rename.groupby("Year")["Total Team Salary"].mean().reset_index()

# Linear regression line
slope, intercept,_,_,_ = linregress(avg_salary_per_year["Year"], avg_salary_per_year["Total Team Salary"])
regression_line = slope * avg_salary_per_year["Year"] + intercept

# Plotting the regression chart
plt.figure(figsize=(10, 6))
plt.scatter(avg_salary_per_year["Year"], avg_salary_per_year["Total Team Salary"], color='blue', label="Average Total Team Salary")
plt.plot(avg_salary_per_year["Year"], regression_line, color="red", linestyle="-", label="Regression Line")
plt.title("Average Total Team Salary per Year (Regression)")
plt.xlabel("Year")
plt.ylabel("Average Total Team Salary, USD")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.tight_layout()
plt.ticklabel_format(style='plain',axis="y")
formatter = ticker.StrMethodFormatter('{x:,.0f}')
plt.gca().yaxis.set_major_formatter(formatter)
plt.savefig("Output/Avg Regression Fig.png")
plt.show

# Question 2 Analysis, wins vs salary ( Wanderson Oliveira)

In [ ]:
#Study data files
team_salary_merged_path = "Output/team_salary_merged.csv"
#Read Study Results
team_salary_merged = pd.read_csv(team_salary_merged_path)
team_salary_merged.head()

In [ ]:
# Most wins from higher to lower
top_wins = team_salary_merged.sort_values(["W"],ascending=False)
top_wins.head(30)

In [ ]:
#Wins selected by year ( All teams)
top_wins_group = top_wins.groupby("yearID").apply(lambda x: x.sort_values('W',ascending=False))

top_wins_group.head()

In [ ]:
# Team with most victories by year 
dataframe = top_wins_group.drop_duplicates(subset=['yearID'])
dataframe

In [ ]:
not_strike= dataframe.copy(deep=True)
not_strike = not_strike[not_strike['yearID']!=1994]
not_strike['log_salary'] = np.log(not_strike['total_player_salaries'])
not_strike.head(30)

In [ ]:
# Calculating Mean of Wins per each team 
summary_aggr =  team_salary_merged.groupby(['franchID'])[['W']].agg(['mean'])
summary_aggr

In [ ]:
#Correlation between wins and Salary over the years
print(not_strike['W'].corr(not_strike['total_player_salaries']))

In [ ]:
# Average of wins over the years.
plt.figure(figsize=(14,10))  ## width, height
team_salary_merged.groupby('franchID')['W'].mean().plot.bar()
plt.xlabel('Teams')
plt.ylabel(' Avg Wins')
plt.title('Average of wins across years')
plt.savefig('Output/team_avg_wins.png')
#plt.legend(bbox_to_anchor=(1.0, 1.0))
plt.show()

In [ ]:
# Assigning names to columns in DF
not_strike_renamed = not_strike.rename(columns={
    'franchID': 'Teams',
    'total_player_salaries': 'Salary, USD',
    'yearID': 'Years',
    'W': 'Wins'
})

In [ ]:
# Correlation between Wins x Salary from 1985 to 2015
plt.figure(figsize=(14, 9))  ## width, height
sns.scatterplot(x='Years', y='Salary, USD', data=not_strike_renamed, size='Wins', hue='Teams')
plt.xlabel('Years')
plt.ylabel('Salary, USD')
plt.title('Team with most wins each season vs team player salaries')
plt.legend(bbox_to_anchor=(1.0, 1.0))
# Formatting y-axis labels to display salary in USD format
formatter = ticker.StrMethodFormatter('{x:,.0f}')
plt.gca().yaxis.set_major_formatter(formatter)
plt.savefig('Output\wins vs salary.png')
plt.show()

# Ouestion 3 Analysis, team performance (Molly Ingram)

In [ ]:
# Relationship btwn performance stats and wins
team_salary_tax = team_salary_tax_df

plt.scatter(team_salary_tax['W'],team_salary_tax['R'])
plt.xlabel('Wins')
plt.ylabel('Runs')
plt.show()

plt.scatter(team_salary_tax['W'],team_salary_tax['SF'])
plt.xlabel('Wins')
plt.ylabel('Scrafice Flies')
plt.show()

plt.scatter(team_salary_tax['W'],team_salary_tax['DP'])
plt.xlabel('Wins')
plt.ylabel('Double Plays')
plt.show()

In [ ]:
#Relationship btwn performance stats and team salary variance
plt.scatter(team_salary_tax['salary_var'],team_salary_tax['R'])
plt.xlabel('Player Salary Variance')
plt.ylabel('Runs')
plt.show()

plt.scatter(team_salary_tax['salary_var'],team_salary_tax['SB'])
plt.xlabel('Player Salary Variance')
plt.ylabel('Stolen Bases')
plt.show()

plt.scatter(team_salary_tax['salary_var'],team_salary_tax['DP'])
plt.xlabel('Player Salary Variance')
plt.ylabel('Double Plays')
plt.show()


In [ ]:
#Relationship btwn performance stats and team salary 
plt.scatter(team_salary_tax['total_player_salaries'],team_salary_tax['HR'])
plt.xlabel('Total Player Salaries')
plt.ylabel('Home Runs')
plt.show()

plt.scatter(team_salary_tax['total_player_salaries'],team_salary_tax['3B'])
plt.xlabel('Total Player Salaries')
plt.ylabel('Triples')
plt.show()

plt.scatter(team_salary_tax['total_player_salaries'],team_salary_tax['E'])
plt.xlabel('Total Player Salaries')
plt.ylabel('Errors')
plt.show()

In [ ]:
team_salary_tax.columns

### Using RandomForest Regressor to find which performance variables best relate to wins and team player salaries

In [ ]:
# Pulling out feature and target variables

rf_df = team_salary_tax[['W','R', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'CS',
                         'RA', 'ER', 'ERA', 'CG','SHO','SV', 'HA', 'HRA', 'BBA',
                           'SOA', 'E', 'DP', 'FP','total_player_salaries']]


rf_df.isnull().sum()

In [ ]:
# Generating training and test samples

x = rf_df.drop(columns=['W', 'total_player_salaries'])
y_wins = rf_df['W']
y_salary = rf_df['total_player_salaries']

from sklearn.model_selection import train_test_split

training_w, testing_w, training_labels_w, testings_labels_w = train_test_split(x, y_wins,test_size = .5, random_state = 42)
training_s, testing_s, training_labels_s, testings_labels_s = train_test_split(x, y_salary,test_size = .5, random_state = 42)

In [ ]:
#Looking at relationships btwn vars
rf_df = rf_df.rename(columns={'total_player_salaries_100k': 'Player Salaries', 'W': "Wins"})

sns.heatmap(rf_df.corr(),xticklabels=rf_df.columns, yticklabels=rf_df.columns)
plt.title('Correlations between Performance Stats, Wins, and Salaries')

plt.savefig('Output/performance correlations.png')


In [ ]:
#Running RF for wins and salary outcomes

from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf_w = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf_s = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf_w.fit(training_w, training_labels_w)
rf_s.fit(training_s, training_labels_s)

#Checking performance 
predictions_w = rf_w.predict(testing_w)
error_w = abs(predictions_w - testings_labels_w)
mape_w = 100 * (error_w / testings_labels_w)
accuracy_w = 100 - np.mean(mape_w)
print('Accuracy in estimating wins:', round(accuracy_w, 2), '%.')

predictions_s = rf_s.predict(testing_s)
error_s = abs(predictions_s - testings_labels_s)
mape_s = 100 * (error_s / testings_labels_s)
accuracy_s = 100 - np.mean(mape_s)
print('Accuracy in estimating salary:', round(accuracy_s, 2), '%.')


In [ ]:
#Feature importance
importances_w = list(rf_w.feature_importances_)
feature_importances_w = [(feature, round(importance, 3)) for feature, importance in zip(x.columns, importances_w)]
# Sort the feature importances by most important first
feature_importances_w = sorted(feature_importances_w, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
print('Variable importance for predicting wins')
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances_w]

print('-----------------------------------------------------------------------------------')

importances_s = list(rf_s.feature_importances_)
feature_importances_s = [(feature, round(importance, 3)) for feature, importance in zip(x.columns, importances_s)]
# Sort the feature importances by most important first
feature_importances_s = sorted(feature_importances_s, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
print('Variable importance for predicting salary')
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances_s]


In [ ]:
#Adding performance index to main df

team_salary_tax['rf_pred'] = rf_w.predict(x)

team_salary_tax.head()

team_salary_tax['W'].corr(team_salary_tax['rf_pred'])

In [ ]:
#Performance prediction vs wins
plt.scatter(team_salary_tax['rf_pred'], team_salary_tax['W'], alpha=.5, edgecolors='black')
plt.xlabel('Wins Predicted by Performance')
plt.ylabel('Actual Wins')

plt.savefig('Output/perdicted vs actual.png')


In [ ]:
# Statistical tests on performance distributions pre- and post- tax rules

perf_pre = team_salary_tax.loc[team_salary_tax['yearID']<1997, ['rf_pred']]
perf_post = team_salary_tax.loc[team_salary_tax['yearID']>=1997, ['rf_pred']]
perf_post2003 = team_salary_tax.loc[team_salary_tax['yearID']>=2003, ['rf_pred']]

#Testing variances
print('F test results for pre and post', stats.f_oneway(perf_pre,perf_post))
print('F test results for pre and post 2003', stats.f_oneway(perf_pre,perf_post2003))

#Testing means (speficially if post is higher)
print('Right tailed T test results for pre and post', stats.ttest_ind(perf_post, perf_pre, alternative='greater'))
print('Right tailed T test results for pre and post 2003', stats.ttest_ind(perf_post2003, perf_pre, alternative='greater'))


fig, (ax1,ax2,ax3) = plt.subplots(3, sharey=True, figsize=(7,6))
ax1.hist(perf_pre,bins=50)
ax1.set_ylabel('1985 - 1996')
ax2.hist(perf_post,bins=50)
ax2.set_ylabel('1997 - 2015')
ax3.hist(perf_post2003, bins=50)
ax3.set_ylabel('2003 - 2015')
fig.suptitle('Team Performance Distributions')
fig.supxlabel('Wins Predicted by Performance')
fig.supylabel('Years')

plt.savefig('Output/perf distributions.png')



In [ ]:
#Generating annual performance predictions
years = team_salary_tax['yearID'].unique()

perf_stats = dict()
for year in years:
    data = team_salary_tax.loc[team_salary_tax['yearID']==year, ['rf_pred']]
    perf_stats[year] = data['rf_pred'].values

perf_stats = pd.DataFrame.from_dict(perf_stats)

plt.figure(figsize=(15,5))
plt.boxplot(perf_stats, positions=np.array(np.arange(len(years))*3), widths=.75)
plt.axvline(34.5, color='r' )
plt.annotate('Taxes based on player salaries begin', xy=[34.75, 50],fontstyle = 'italic')
plt.xticks(np.arange(0,len(years)*3,3),years, rotation = 90)
plt.xlim(-3,len(years)*3)
plt.ylabel('Wins Predicted by Performance')
plt.xlabel('Year')
plt.title('Performance Distributions')
plt.savefig('Output/Performance by year.png')

plt.show()
